In [15]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import pickle
from IPython.display import display
#evaluations
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

from sklearn.metrics import roc_curve 
from sklearn.metrics import roc_auc_score 
from sklearn.model_selection import train_test_split

# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [5]:
count = 0 
def func() :
    global count 
    count=count+1 
func()
count 

1

In [8]:
cd D:\1MYFILES\ML_NGFW\FIREWALLS\mangyaWAF\models

D:\1MYFILES\ML_NGFW\FIREWALLS\mangyaWAF\models


In [4]:
data = pd.read_csv("https://trello-attachments.s3.amazonaws.com/5ed2d4107c349c221194b608/5ed2d453f0e5a45bcd8cf16c/9d9eab0bcf8afdfbdff5fdb173f78cd6/final-allpayload.csv",index_col="index")

Y = data['is_malicious']
"""independent_variables = data.columns
independent_variables = independent_variables.delete(1)
independent_variables"""
X = data.iloc[:,3:]

from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2)

In [83]:
def get_thresh_accu_df( model ) : 
    i = 1 
    print( "--------------------------",model,"--------------------------")
    pred=[]
    # appending all the probabilities that are classified as bad url 
    pred.append(pd.Series(model.predict_proba(testX)[:,1]))
    # getting the mean probabilities of all the ensembling models for bad url 
    final_prediction=pd.concat(pred,axis=1).mean(axis=1)
    print(model,': test roc-auc: {}'.format(roc_auc_score(testY,final_prediction)))
    
    # passing the data & probabilites to get thresholds, fpr & tpr 
    fpr, tpr, thresholds = roc_curve(testY, final_prediction)
    print( model, ": ", fpr )
    from sklearn.metrics import accuracy_score
    accuracy = []
    for thres in thresholds:
        # marking the bad url based on different threshold values available 
        y_pred = np.where(final_prediction>thres,1,0)
        # and finding the accuracy based on the thresholds 
        accuracy.append(accuracy_score(testY, y_pred, normalize=True))

    accuracy_ls = pd.concat([pd.Series(thresholds), pd.Series(accuracy)],
                            axis=1) 
    accuracy_ls.columns = ['thresholds', 'accuracy']
    accuracy_ls.sort_values(by='accuracy', ascending=False, inplace=True)
    print('accuracy specific threshold : \n',accuracy_ls.head(1))
    return( accuracy_ls ) 

    # OPTIMAL THRESHOLD USING GMEAN 
    
#     y_pred = model.predict_proba( testX ) 
#     y_pred = y_pred[:,1]
    
#     fpr, tpr, thresholds = roc_curve( testY,y_pred ) 
#     gmean = np.sqrt(tpr * (1-fpr))
#     # Find the optimal threshold
#     index = np.argmax(gmean)
#     thresholdOpt = round(thresholds[index], ndigits = 4)
#     gmeanOpt = round(gmean[index], ndigits = 4)
#     fprOpt = round(fpr[index], ndigits = 4)
#     tprOpt = round(tpr[index], ndigits = 4)
#     print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
#     print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))


#     # OPTIMAL THRESHOLD USING YOUDEN J  
#     fpr, tpr, thresholds = roc_curve( testY,y_pred )
#     youdenJ = tpr - fpr
#     index = np.argmax(youdenJ)
#     thresholdOpt = round(thresholds[index], ndigits = 4)
#     youdenJOpt = round(gmean[index], ndigits = 4)
#     fprOpt = round(fpr[index], ndigits = 4)
#     tprOpt = round(tpr[index], ndigits = 4)
#     print('Best Threshold: {} with Youden J statistic: {}'.format(thresholdOpt, youdenJOpt))
#     print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

#     # Find the optimal threshold
#     precision, recall, thresholds = precision_recall_curve(testY, y_pred)
#     fscore = (2 * precision * recall) / (precision + recall)
#     index = np.argmax(fscore)
#     thresholdOpt = round(thresholds[index], ndigits = 4)
#     fscoreOpt = round(fscore[index], ndigits = 4)
#     recallOpt = round(recall[index], ndigits = 4)
#     precisionOpt = round(precision[index], ndigits = 4)
#     print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))
#     print('Recall: {}, Precision: {}'.format(recallOpt, precisionOpt))
    
#     print("----------------------------------------------------")

# # accuracy specific threshold 
# # accuracy 
# # roc_auc_score 

In [84]:
#     y_pred = lr_classifier.predict( testX ) 
#     y_pred

In [85]:
def gmean_thresh( model ) :
    y_pred = model.predict_proba( testX ) 
    y_pred = y_pred[:,1]
    
    fpr, tpr, thresholds = roc_curve( testY,y_pred ) 
    gmean = np.sqrt(tpr * (1-fpr))
    # Find the optimal threshold
    index = np.argmax(gmean)
    thresholdOpt = round(thresholds[index], ndigits = 4)
#     accuracyOpt = round( accuracy_ls.accuracy[index], ndigits = 4)
    gmeanOpt = round(gmean[index], ndigits = 4)
    fprOpt = round(fpr[index], ndigits = 4)
    tprOpt = round(tpr[index], ndigits = 4)
    print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
#     print('accuracy : ',accuracyOpt)
    print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))
    
    
def youden_thresh( model ) : 
    y_pred = model.predict_proba( testX ) 
    y_pred = y_pred[:,1]
    
    fpr, tpr, thresholds = roc_curve( testY,y_pred ) 
    # Find the optimal threshold
    youdenJ = tpr - fpr
    gmean = np.sqrt(tpr * (1-fpr))
    index = np.argmax(youdenJ)
    thresholdOpt = round(thresholds[index], ndigits = 4)
#     accuracyOpt = round( accuracy_ls.accuracy[index], ndigits = 4)
    youdenJOpt = round(gmean[index], ndigits = 4)
    fprOpt = round(fpr[index], ndigits = 4)
    tprOpt = round(tpr[index], ndigits = 4)
    print('Best Threshold: {} with Youden J statistic: {}'.format(thresholdOpt, youdenJOpt))
#     print('accuracy : ',accuracyOpt)
    print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))
    

def precision_thresh( model ) : 
    
    y_pred = model.predict_proba( testX ) 
    y_pred = y_pred[:,1]
    
    precision, recall, thresholds = precision_recall_curve(testY, y_pred)    # Find the optimal threshold
    fscore = (2 * precision * recall) / (precision + recall)
    index = np.argmax(fscore)
    thresholdOpt = round(thresholds[index], ndigits = 4)
#     accuracyOpt = round( accuracy_ls.accuracy[index], ndigits = 4)
    fscoreOpt = round(fscore[index], ndigits = 4)
    recallOpt = round(recall[index], ndigits = 4)
    precisionOpt = round(precision[index], ndigits = 4)
    print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))
#     print('accuracy : ',accuracyOpt)
    print('Recall: {}, Precision: {}'.format(recallOpt, precisionOpt))
    print("--------------------------------------------------------------------------\n\n")

In [170]:
                xgb_classifier = pickle.load(open("xgb_classifier.pkl", "rb")) 
                ada_classifier = pickle.load(open("ada_classifier.pkl", "rb")) 
                sgd_classifier = pickle.load(open("sgd_classifier.pkl", "rb")) 
                lr_classifier = pickle.load(open("lr_classifier.pkl", "rb")) 
                rf_classifier = pickle.load(open("rf_classifier.pkl", "rb")) 
                #svm_classifier = pickle.load(open("svm_classifier.pkl", "rb")) 
                dt_classifier = pickle.load(open("dt_classifier.pkl", "rb")) 
                mnb_classifier = pickle.load(open("mnb_classifier.pkl", "rb")) 

In [9]:
from sklearn.svm import SVC
svm_classifier = SVC( probability = True )

Y = data['is_malicious']
"""independent_variables = data.columns
independent_variables = independent_variables.delete(1)
independent_variables"""
X = data.iloc[:,3:]

svm_classifier.fit( X, Y )

SVC(probability=True)

In [87]:
for model in [xgb_classifier,ada_classifier,lr_classifier,rf_classifier,svm_classifier,dt_classifier,mnb_classifier]:
    accuracy = get_thresh_accu_df( model )
    gmean_thresh( model )
    youden_thresh( model  ) 
    precision_thresh( model  ) 

-------------------------- XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...) --------------------------
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric

In [ ]:
accuracy specific threshold 
accuracy 
roc_auc_score 

best threshold 
gmean 
optimal fpr
optimal tpr 

best threshold 
optimal fscore 
optimal recall 
optimal precison

AUC OF ROC 
AUC OF PR CURVE 

# MODELS 

In [139]:
from sklearn.ensemble import AdaBoostClassifier
ada_classifier = AdaBoostClassifier()

from sklearn.linear_model import SGDClassifier
sgd_classifier = SGDClassifier() 

from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(max_iter = 110359) 

from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier() 

from sklearn.svm import SVC
svm_classifier = SVC() 

from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier() 

from sklearn.naive_bayes import MultinomialNB
mnb_classifier = MultinomialNB() 

In [140]:
data = pd.read_csv("https://trello-attachments.s3.amazonaws.com/5ed2d4107c349c221194b608/5ed2d453f0e5a45bcd8cf16c/9d9eab0bcf8afdfbdff5fdb173f78cd6/final-allpayload.csv",index_col="index")

In [141]:
Y = data['is_malicious']
"""independent_variables = data.columns
independent_variables = independent_variables.delete(1)
independent_variables"""
X = data.iloc[:,3:]

In [142]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2)    

In [143]:
xgb_classifier.fit(X,Y)
ada_classifier.fit(X,Y)
sgd_classifier.fit(X,Y)
lr_classifier.fit(X,Y)
rf_classifier.fit(X,Y)
svm_classifier.fit(X,Y)
dt_classifier.fit(X,Y)
mnb_classifier.fit(X,Y)

MultinomialNB()

### FINDING OPTIMIZED PRECISION & ACCURACY 

In [104]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [127]:
dec = lr_classifier.decision_function( testX )

In [128]:
for i in dec : 
    print( i ) 

-23.51340385835005
-23.413292081343034
-6.948737731601785
-6.285391605794647
-23.32064180466698
-8.178536265795525
-15.433744483513326
-7.171212042625598
-22.664023166110372
-22.910809720654424
-5.738080683881623
11.09474735521436
14.004368932609493
-23.44283325527555
-14.208427903001997
-23.26752077941892
-12.53665578367373
-22.652047585036254
-22.557813805012447
-5.731116660484076
-23.553926053438804
-23.39801752398458
-24.039484511942216
-2.926122665266078
-5.7907573646867725
-1.9341953458320056
-24.171248968798476
1.1718990800016176
-23.7223521790735
-2.36036792480318
-23.62109073713324
-23.96422857026859
-1.1389138020458844
-23.697755953673674
-23.39370168156601
-23.463034150146445
-3.9614089885310992
-24.94912205835799
-7.024956680936693
-22.549204143916217
-23.38468355065019
-23.97947876436906
-24.442130428860526
-27.421402781674505
-19.830897069442806
2.730200026831584
-7.744293757618301
-6.740638297980851
-6.735572557511897
-4.263472820181628
-4.854959069503973
-3.525512024602

In [129]:
prob = lr_classifier.predict(testX)
for i in prob : 
    print( i )

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [131]:
prob = lr_classifier.predict_proba(testX)
prob = prob[:,1]
for i in prob : 
    print( i )

6.141273705407822e-11
6.787915789623477e-11
0.0009589255517535299
0.0018598616065771106
7.446873040638704e-11
0.00028053366038940995
1.9824847242414048e-07
0.0007678008729806072
1.4359515976505687e-10
1.1219196700020486e-10
0.003210603182701468
0.9999848083184599
0.999999172096935
6.590325679775535e-11
6.75084088628875e-07
7.853154069866342e-11
3.592510212783702e-06
1.4532513328627346e-10
1.596856684521093e-10
0.0032329673954534757
5.897390526531716e-11
6.892394097311121e-11
3.6289796149665074e-11
0.05087723082120494
0.0030463583885831274
0.12628694550207276
3.180972775010168e-11
0.7634881118044528
4.983257749908747e-11
0.08624519504588456
5.514303130250786e-11
3.9126208854429816e-11
0.2425198440619863
5.107346887106896e-11
6.922204867139721e-11
6.458530871920396e-11
0.018680663280542884
1.461281754414798e-11
0.0008886157068244469
1.6106644341211647e-10
6.984912544893669e-11
3.853405329496286e-11
2.4261498914689437e-11
1.2332072971409332e-12
2.4409047385430993e-09
0.9387853332172604
0.

# PRECISION OPTIMIZATION USING HYPER PLANES

In [171]:

def optimal_params( model , ThreshOpt ) : 
    df = model.decision_function(testX) 
    desired_predict = []

    # Iterate through each value of decision function output
    # and if  decision score is > than Decision threshold then,
    # append (1) to the empty list ( desired_prediction) else
    # append (0).
    print("\n",model,"\n")
    for i in df:
        if i<ThreshOpt:
            desired_predict.append(0)
        else:
            desired_predict.append(1)
    y_pred = model.predict( testX ) 
    print("\n------------------ Threshold : ", ThreshOpt,"--------------------\n")
    
    old_prec = precision_score(testY, y_pred)
    new_prec = precision_score(testY, desired_predict)
    
    print("old precision value:", old_prec)
    print("new precision value:", new_prec)
    
    print("\nold accuracy value:", accuracy_score(testY, y_pred))
    print("new accuracy value:", accuracy_score(testY, desired_predict))

    old_recall = recall_score(testY, y_pred)
    new_recall = recall_score(testY, desired_predict)
    
    print("\nold recall value:", recall_score(testY, y_pred))
    print("new recall value:", recall_score(testY, desired_predict))
   
    old_f1 = (2 * ( old_prec * old_recall )) / ( old_prec + old_recall )
    new_f1 = (2 * ( new_prec * new_recall )) / ( new_prec + new_recall )
    
    print("\nold f1 score:", recall_score(testY, y_pred))
    print("new f1 score:", recall_score(testY, desired_predict))
   
    # roc_auc_score(test_data , predict_proba ) 
    

In [172]:
thresholds =  [
#     [0.33242,0.329400003,0.448599994], 
    [0.496494,0.4839,0.4965], 
#     [0.425976,0.175,0.426],
    [0.37,0.59,0.59],
    [0.343167,0.1493,0.3425],
#     [0,1,1],
    [1,0.5309,1]
]

# rf_classifier,dt_classifier,

In [173]:
for model in [ada_classifier,lr_classifier,svm_classifier,mnb_classifier]:
    i = 0 
    j = 0 
    optimal_params( model , thresholds[i][j])
    j += 1 
    optimal_params( model , thresholds[i][j])
    j += 1 
    optimal_params( model , thresholds[i][j])
    i += 1 
    j = 0 


 AdaBoostClassifier() 


------------------ Threshold :  0.496494 --------------------

old precision value: 0.9484083424807903
new precision value: 0.0

old accuracy value: 0.9874954693729612
new accuracy value: 0.9134650235592606

old recall value: 0.9047120418848168
new recall value: 0.0

old f1 score: 0.9047120418848168
new f1 score: 0.0


C:\Users\lanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
C:\Users\lanka\AppData\Local\Temp\ipykernel_4972\97043195.py:34: RuntimeWarning: invalid value encountered in double_scalars



 AdaBoostClassifier() 


------------------ Threshold :  0.4839 --------------------

old precision value: 0.9484083424807903
new precision value: 0.0

old accuracy value: 0.9874954693729612
new accuracy value: 0.9134650235592606

old recall value: 0.9047120418848168
new recall value: 0.0

old f1 score: 0.9047120418848168
new f1 score: 0.0


C:\Users\lanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
C:\Users\lanka\AppData\Local\Temp\ipykernel_4972\97043195.py:34: RuntimeWarning: invalid value encountered in double_scalars



 AdaBoostClassifier() 


------------------ Threshold :  0.4965 --------------------

old precision value: 0.9484083424807903
new precision value: 0.0

old accuracy value: 0.9874954693729612
new accuracy value: 0.9134650235592606

old recall value: 0.9047120418848168
new recall value: 0.0

old f1 score: 0.9047120418848168


C:\Users\lanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
C:\Users\lanka\AppData\Local\Temp\ipykernel_4972\97043195.py:34: RuntimeWarning: invalid value encountered in double_scalars


new f1 score: 0.0

 LogisticRegression(max_iter=110359) 


------------------ Threshold :  0.496494 --------------------

old precision value: 0.9542824074074074
new precision value: 0.9780701754385965

old accuracy value: 0.9845958680681406
new accuracy value: 0.9826023921710765

old recall value: 0.8633507853403142
new recall value: 0.8172774869109948

old f1 score: 0.8633507853403142
new f1 score: 0.8172774869109948

 LogisticRegression(max_iter=110359) 


------------------ Threshold :  0.4839 --------------------

old precision value: 0.9542824074074074
new precision value: 0.9781659388646288

old accuracy value: 0.9845958680681406
new accuracy value: 0.9829195360637912

old recall value: 0.8633507853403142
new recall value: 0.8209424083769633

old f1 score: 0.8633507853403142
new f1 score: 0.8209424083769633

 LogisticRegression(max_iter=110359) 


------------------ Threshold :  0.4965 --------------------

old precision value: 0.9542824074074074
new precision value: 0.978070175

AttributeError: 'MultinomialNB' object has no attribute 'decision_function'

In [ ]:
recall score is swapped with precision score in confusion matrix 

In [ ]:
https://towardsdatascience.com/how-to-add-decision-threshold-tuning-to-your-end-to-end-ml-pipelines-7077b82b71a
    
https://www.analyticsvidhya.com/blog/2020/10/improve-class-imbalance-class-weights/
    
OOB SCORE 

https://github.com/krishnaik06/Gaussian-Trnasformaion/blob/master/Untitled3.ipynb
    
    